## Cargar y parsear archivo

In [2]:
import pandas as pd

In [1]:
url = 'https://raw.githubusercontent.com/AnaLuisaIA/antifungal-peptides/refs/heads/main/databases/Antifungipept/clf_antifungal.csv'

In [3]:
df = pd.read_csv(url)

In [5]:
display(df.head())

,Sequence,Antifungal,Split
0,AAAAGSCVWGAVNYTSDCAAECKRRGYKGGHCGSFANVNCWCET,True,test
1,AAAAGSCVWGAVNYTSDCAAECKRRGYKGGHCGSFANVNCWCRT,True,train
2,AAAAGSCVWGAVNYTSDCAAECLLRGYKGGHCGSFANVNCWCET,True,train
3,AAAAGSCVWGAVNYTSDCAAECLLRGYKGGHCGSFANVNCWCRT,True,train
4,AAAAGSCVWGAVNYTSDCNGECKRRGYKGGHCGSFANVNCWCET,True,train


In [6]:
df.shape

(11550, 3)

In [12]:
df["Antifungal"] = df["Antifungal"].astype(int)
print(df.head())

                                       Sequence  Antifungal  Split
0  AAAAGSCVWGAVNYTSDCAAECKRRGYKGGHCGSFANVNCWCET           1   test
1  AAAAGSCVWGAVNYTSDCAAECKRRGYKGGHCGSFANVNCWCRT           1  train
2  AAAAGSCVWGAVNYTSDCAAECLLRGYKGGHCGSFANVNCWCET           1  train
3  AAAAGSCVWGAVNYTSDCAAECLLRGYKGGHCGSFANVNCWCRT           1  train
4  AAAAGSCVWGAVNYTSDCNGECKRRGYKGGHCGSFANVNCWCET           1  train


In [8]:
df_antifungal = df[df["Antifungal"] == 1]
df_no_antifungal = df[df["Antifungal"] == 0]

In [9]:
df_antifungal.shape

(5775, 3)

In [10]:
df_no_antifungal.shape

(5775, 3)

## Verificar duplicados

In [19]:
df["Sequence"].unique().shape

(11550,)

In [14]:
df_antifungal = df_antifungal.drop_duplicates().copy()

In [15]:
df_antifungal.shape

(5775, 3)

In [16]:
df_no_antifungal["Sequence"].unique().shape

(5775,)

In [17]:
df_no_antifungal = df_no_antifungal.drop_duplicates().copy()

In [18]:
df_no_antifungal.shape

(5775, 3)

## Verificar canónicos

In [20]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [22]:
# Verificar cuáles secuencias válidas
df["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df["Sequence"]]

In [23]:
no_validas = df[~df["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 0


In [25]:
df_validas = df[df["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 11550


In [26]:
df_validas = df_validas.rename(columns={"Sequence": "sequence", "Antifungal": "label"})

In [27]:
df_validas = df_validas.drop(columns=["Split"])

In [29]:
df_validas.to_csv('AFP_labeled.csv', index=False, header=True)

## Obtener metadatos

In [30]:
import json

In [31]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [39]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [33]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [43]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "Antifungipept")

In [44]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df_validas)

In [45]:
export_json("metadata.json", dict_metadata)